In [17]:
import random
import numpy as np
import matplotlib.pyplot as plt


import torch
import torch.nn as nn


import torch.optim as optim 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
# Set up for getting reproducibility of result
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [20]:
# Hyperparameters
n_classes = 10
learning_rate = 0.001
batch_size = 64
n_epochs = 50

In [22]:
# Load 'MNIST' dataset
train_dataset = datasets.CIFAR10(root='/content/drive/MyDrive/Datasets/CIFAR10', train=True, 
                                                    transform=transforms.ToTensor(), download=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root='/content/drive/MyDrive/Datasets/CIFAR10', train=False, 
                                                  transform=transforms.ToTensor(), download=False)

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [23]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /content/drive/MyDrive/Datasets/CIFAR10
    Split: Train
    StandardTransform
Transform: ToTensor()

In [24]:
test_dataset

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: /content/drive/MyDrive/Datasets/CIFAR10
    Split: Test
    StandardTransform
Transform: ToTensor()

In [25]:
class LeNet5(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.Flatten(),
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
            nn.Softmax(dim=1)
        )
    def forward(self, X):
        prob = self.model(X)
        return prob

In [26]:
model = LeNet5(n_classes=n_classes).to(device)
model

LeNet5(
  (model): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): Tanh()
    (11): Linear(in_features=84, out_features=10, bias=True)
    (12): Softmax(dim=1)
  )
)

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [28]:
for epoch in range(n_epochs):
    for batch_idx, (images, labels) in enumerate(train_loader):
        # Get data to GPU
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        z_scores = model(images)
        loss = criterion(z_scores, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent
        optimizer.step()

        if(batch_idx + 1) % 100 == 0:
            print(f'Epoch {epoch+1}/{n_epochs}, Batch {batch_idx+1}, Loss: {loss.item():.4f}')

Epoch 1/50, Batch 100, Loss: 2.1772
Epoch 1/50, Batch 200, Loss: 2.1505
Epoch 1/50, Batch 300, Loss: 2.1632
Epoch 1/50, Batch 400, Loss: 2.0635
Epoch 1/50, Batch 500, Loss: 2.1292
Epoch 1/50, Batch 600, Loss: 1.9707
Epoch 1/50, Batch 700, Loss: 2.1253
Epoch 2/50, Batch 100, Loss: 2.0021
Epoch 2/50, Batch 200, Loss: 2.0485
Epoch 2/50, Batch 300, Loss: 2.1417
Epoch 2/50, Batch 400, Loss: 2.0870
Epoch 2/50, Batch 500, Loss: 1.9966
Epoch 2/50, Batch 600, Loss: 2.1462
Epoch 2/50, Batch 700, Loss: 2.0172
Epoch 3/50, Batch 100, Loss: 1.9908
Epoch 3/50, Batch 200, Loss: 2.0238
Epoch 3/50, Batch 300, Loss: 1.9441
Epoch 3/50, Batch 400, Loss: 2.0661
Epoch 3/50, Batch 500, Loss: 2.1222
Epoch 3/50, Batch 600, Loss: 2.0073
Epoch 3/50, Batch 700, Loss: 1.9918
Epoch 4/50, Batch 100, Loss: 1.9851
Epoch 4/50, Batch 200, Loss: 2.0575
Epoch 4/50, Batch 300, Loss: 1.9538
Epoch 4/50, Batch 400, Loss: 1.9675
Epoch 4/50, Batch 500, Loss: 2.0576
Epoch 4/50, Batch 600, Loss: 1.9670
Epoch 4/50, Batch 700, Loss:

In [29]:
def get_accuracy(loader, model):
    if loader.dataset.train:
        print('Getting accuracy on trainning data')
    else:
        print('Getting accuracy on testing data')

    n_corrects = 0
    n_samples = 0

    # Put model to evalution mode
    model.eval()

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            z_scores = model(images)

            _, y_pred = z_scores.max(1)
            n_corrects += (y_pred == labels).sum()
            n_samples += y_pred.size(0)

    print(f'We got {n_corrects}/{n_samples} correct')
    print(f'Accuracy =  {float(n_corrects) / float(n_samples)*100.0:.2f}')
    # Put model to train mode
    model.train()

In [30]:
get_accuracy(train_loader, model)
get_accuracy(test_loader, model)

Getting accuracy on trainning data
We got 36548/50000 correct
Accuracy =  73.10
Getting accuracy on testing data
We got 5284/10000 correct
Accuracy =  52.84
